In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Activation, BatchNormalization
from keras.models import Model
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

In [7]:
path_train = '../../../data/compstat2/train.csv'
path_test = '../../../data/compstat2/test.csv'
random_state = 2018
epochs = 50
batch_size = 128

In [1]:
def load_train():
    data = pd.read_csv(
        path_train, delimiter=',',
        dtype='|U', encoding='utf-8'
    ).values
    x = data[:, :-1].astype(dtype=float)
    y = data[:, -1].astype(dtype=float)
    x_train, x_val, y_train, y_val = train_test_split(
        x, y, test_size=0.05, random_state=random_state
    )
    return (x_train, y_train), (x_val, y_val)

In [2]:
def load_pred():
    data = pd.read_csv(
        path_test, delimiter=',',
        dtype='|U', encoding="utf-8"
    ).values
    x_id = data[:, -1]
    x = data[:, :-1]
    return (x_id.astype(dtype=int), x.astype(dtype=float))

In [14]:
def preprocessing():
    (x_train, y_train), (x_val, y_val) = load_train()
    (x_id, x_test) = load_pred()

    x_train = x_train.astype('float32')
    x_val = x_val.astype('float32')
    x_test = x_test.astype('float32')

    x_train = x_train / 255
    x_val = x_val / 255
    x_test = x_test / 255
    y_train = to_categorical(y_train)
    y_val = to_categorical(y_val)

    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_val = x_val.reshape(x_val.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_val.shape[0], 'validation samples')

    return (x_train, y_train), (x_val, y_val), (x_id, x_test)

In [15]:
def base_model():

    inputs = Input(shape=(28, 28, 1))

    x = Conv2D(16, (3, 3),
               padding='same',
               kernel_regularizer='l2',
               kernel_initializer='lecun_normal')(inputs)
    x = Activation(activation='selu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)

    x = Conv2D(32, (3, 3),
               padding='same',
               kernel_regularizer='l2',
               kernel_initializer='lecun_normal')(x)
    x = Activation(activation='selu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)

    x = Conv2D(32, (3, 3),
               padding='same',
               kernel_regularizer='l2',
               kernel_initializer='lecun_normal')(x)
    x = Activation(activation='selu')(x)
    x = BatchNormalization()(x)

    x = Conv2D(32, (3, 3),
               padding='same',
               kernel_regularizer='l2',
               kernel_initializer='lecun_normal')(x)
    x = Activation(activation='selu')(x)
    x = BatchNormalization()(x)

    x = Conv2D(64, (3, 3), kernel_initializer='lecun_normal')(x)
    x = Activation(activation='selu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)

    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation(activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(512)(x)
    x = Activation(activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(102)(x)
    y = Activation('softmax')(x)
    model = Model(inputs=inputs, outputs=y)
    return model

In [20]:
(x_train, y_train), (x_val, y_val), (x_id, x_test) = preprocessing()

(6650, 102)
x_train shape: (6650, 28, 28, 1)
6650 train samples
350 validation samples


In [24]:
model = base_model()
model.summary()
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 16)        160       
_________________________________________________________________
activation_1 (Activation)    (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
activation_2 (Activation)    (None, 14, 14, 32)        0         
__________

In [25]:
# data augmentations
train_gen = ImageDataGenerator(
    rotation_range=8,
    width_shift_range=0.08,
    shear_range=0.3,
    height_shift_range=0.08,
    zoom_range=0.08
)
val_gen = ImageDataGenerator()
train_generator = train_gen.flow(x_train, y_train, batch_size=batch_size)
val_generator = val_gen.flow(x_val, y_val, batch_size=batch_size)

In [29]:
# callbacks
tensorboard = TensorBoard(
    log_dir='./Graph', histogram_freq=0,
    write_graph=True, write_images=True
)
early_stop = EarlyStopping(
    monitor='val_loss', min_delta=0.0001,
    patience=2, verbose=1, mode='auto'
)
best_model = ModelCheckpoint(
    'caltech101.h5', save_best_only=True, verbose=0
)
plateau = ReduceLROnPlateau(
    factor=np.sqrt(0.1), cooldown=0,
    patience=5, min_lr=0.5e-6
)

In [31]:
# training
model.fit_generator(
    train_generator,
    steps_per_epoch=x_train.shape[0] // 64,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=x_val.shape[0] // 64,
    callbacks=[tensorboard, early_stop, best_model, plateau]
)
train_results = model.evaluate(x_train, y_train)
val_results = model.evaluate(x_val, y_val)
print('train_results acc: ', train_results[1])
print('val_results adc:   ', val_results[1])

Epoch 1/50
103/103 [==============================] - 16s 153ms/step - loss: 2.2441 - acc: 0.5775 - val_loss: 6.0025 - val_acc: 0.0083
Epoch 2/50
103/103 [==============================] - 18s 171ms/step - loss: 1.8358 - acc: 0.6372 - val_loss: 6.3371 - val_acc: 0.0099
Epoch 3/50
103/103 [==============================] - 18s 171ms/step - loss: 1.5859 - acc: 0.6700 - val_loss: 7.4138 - val_acc: 0.0314
Epoch 00003: early stopping
350/350 [==============================] - 0s 445us/step
train_results acc:  0.0267669172932
val_results adc:    0.0314285714286


In [ ]:
y_ = model.predict(x_test)
y_ = np.argmax(y_, axis=1)

x_id = x_id.reshape(x_id.shape[0], 1)
y_ = y_.reshape(y_.shape[0], 1)
results = np.concatenate((x_id, y_), axis=1)
submission_path = os.path.join(
    os.path.dirname(__file__),
    'data/submission.csv')
np.savetxt(submission_path, results, '%d', delimiter=',')